In [17]:
documents = [
"Rafael Nadal Joins Roger Federer in Missing U.S. Open",
"Rafael Nadal Is Out of the Australian Open",
"Biden Announces Virus Measures",
"Biden's Virus Plans Meet Reality",
"Where Biden's Virus Plan Stands"
]

import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from gensim import corpora
from gensim.models import LdaModel

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower()) 
    tokens = [token for token in tokens if token.isalnum()] 
    tokens = [token for token in tokens if token not in stop_words] 
    tokens = [lemmatizer.lemmatize(token) for token in tokens] 
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]
preprocessed_documents

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafael', 'nadal', 'australian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

In [27]:
import pandas as pd

from gensim import corpora

dictionary = corpora.Dictionary(preprocessed_documents)

corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)
# Empty list to store dominant topic labels for each document

article_labels = []

# Iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    # For each document, convert to bag-of-words representation
    bow = dictionary.doc2bow(doc)
    # Get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # Determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # Append to the list
    article_labels.append(dominant_topic)

# Create DataFrame
df = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df)
print()

Table with Articles and Topic:
                                             Article  Topic
0  Rafael Nadal Joins Roger Federer in Missing U....      1
1         Rafael Nadal Is Out of the Australian Open      1
2                     Biden Announces Virus Measures      1
3                   Biden's Virus Plans Meet Reality      1
4                    Where Biden's Virus Plan Stands      1



In [33]:
print ("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"-{word.strip()} (weight: {weight.strip()})")

    print()

Top Terms for Each Topic:
topic 0:
-"biden" (weight: 0.063)
-"stand" (weight: 0.063)
-"virus" (weight: 0.063)
-"plan" (weight: 0.063)
-"announces" (weight: 0.063)
-"measure" (weight: 0.063)
-"australian" (weight: 0.063)
-"nadal" (weight: 0.062)
-"rafael" (weight: 0.062)
-"open" (weight: 0.062)

topic 1:
-"virus" (weight: 0.110)
-"biden" (weight: 0.110)
-"open" (weight: 0.078)
-"rafael" (weight: 0.078)
-"nadal" (weight: 0.078)
-"plan" (weight: 0.078)
-"federer" (weight: 0.047)
-"join" (weight: 0.047)
-"missing" (weight: 0.047)
-"roger" (weight: 0.047)

